# Section 1: Building a Token Information Dashboard

In this section, you'll learn to:
- Fetch a list of tokens with preset conditions
- Get real-time token prices
- Access token market data
- Create interactive price charts
- Display recent transactions

## APIs Used in This Section

### REST API Endpoints:
- **`GET /defi/tokenlist`** - Fetch filtered token list sorted by 24h volume
- **`GET /defi/price`** - Get current token price in real-time
- **`GET /defi/v3/token/market-data`** - Retrieve comprehensive token market data and statistics
- **`GET /defi/history_price`** - Access historical price data for charting
- **`GET /defi/v3/token/txs`** - Get recent token transactions and trading activity

Let's build a comprehensive token information dashboard!

## Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import time

# Import our custom utilities and API wrapper
from utils import (
    BirdeyeDataServices,
    format_currency,
    create_price_chart,
    format_transaction_data,
    check_api_key
)

print("📦 Libraries imported successfully!")

## API Key Setup

First, let's set up our Birdeye Data Services API connection:

In [ ]:
# Initialize Birdeye Data Services with Standard API key
birdeye = BirdeyeDataServices('standard')

# Test API connection
if check_api_key():
    print("✅ API key is valid and ready to use!")
    print("🔗 Connected to Birdeye Data Services")
    print("📊 Ready to fetch token data!")
else:
    print("❌ API key validation failed!")
    print("Please make sure your .env file contains: BDS_STANDARD_API_KEY=your_api_key_here")

### 🔧 Troubleshooting Note

**If you encounter any import errors:**
1. **Restart the kernel**: Go to `Kernel` → `Restart` in the menu
2. **Re-run the import cells above**
3. **Check your .env file** contains the correct API key

All required functions are available in the `utils.py` module.

## Step 1: Discover Listed Tokens With Conditions

Let's start by fetching the tokens with top 24h volume:

In [ ]:
# Fetch top tokens by 24h volume
print("Fetching top tokens by 24h volume...")
token_list = birdeye.get_token_list(
    limit=5, 
    min_liquidity=100000, 
    max_liquidity=10000000, 
    sort_by="v24hUSD", 
    sort_type="desc"
)

if token_list and 'data' in token_list:
    # Handle different possible response structures
    if 'tokens' in token_list['data']:
        tokens_data = token_list['data']['tokens']
    elif 'items' in token_list['data']:
        tokens_data = token_list['data']['items']
    else:
        tokens_data = token_list['data']
    
    tokens_df = pd.DataFrame(tokens_data)
    
    # Display formatted table with available fields
    # Available fields: address, symbol, name, liquidity, v24hUSD, mc
    display_tokens = tokens_df[['name', 'symbol', 'address', 'liquidity', 'v24hUSD', 'mc']].copy()
    display_tokens['Address'] = display_tokens['address'].apply(lambda x: x if x else 'N/A')
    display_tokens['Liquidity'] = display_tokens['liquidity'].apply(lambda x: format_currency(x) if x else 'N/A')
    display_tokens['24h Volume'] = display_tokens['v24hUSD'].apply(lambda x: format_currency(x) if x else 'N/A')
    display_tokens['Market Cap'] = display_tokens['mc'].apply(lambda x: format_currency(x) if x else 'N/A')
    
    final_display = display_tokens[['name', 'symbol', 'Address', 'Liquidity', '24h Volume', 'Market Cap']]
    
    print(f"\nFound {len(tokens_df)} tokens sorted by 24h volume:")
    display(final_display.head(5))
    
    # Store for later use
    selected_tokens = tokens_df.head(5)
    print("\nToken data loaded successfully!")
else:
    print("Could not fetch token list. Please check your API connection.")

## Step 2: Select and Analyze a Token

Now let's pick a token from the list and dive deep into its data:

In [ ]:
# Let's select the first token for detailed analysis
token_address = "So11111111111111111111111111111111111111112"
token_name = "Solana"
token_symbol = "SOL"
print(f"🎯 Selected Token: {token_name} ({token_symbol})")
print(f"📍 Address: {token_address}")
print("-" * 50)

# Helper function to get default token address
def get_default_token_address():
    """Get the selected token address or fallback to SOL"""
    try:
        if 'token_address' in globals() and token_address:
            return token_address
        elif 'selected_tokens' in globals() and not selected_tokens.empty:
            return selected_tokens.iloc[0]['address']
    except:
        pass
    return 'So11111111111111111111111111111111111111112'  # SOL fallback

## Step 3: Get Current Token Price

Let's fetch the current price and basic information. You can either use a token from the list above or enter your own token address:

In [ ]:
# Token address input widget
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create text input for token address
token_address_input = widgets.Text(
    value=get_default_token_address(),  # Use selected token or fallback to SOL
    placeholder='Enter token address',
    description='Token Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Create button to fetch price
fetch_price_button = widgets.Button(
    description='Get Price',
    button_style='info',
    tooltip='Click to fetch current price'
)

# Output widget for results
price_output = widgets.Output()

def fetch_price(b):
    with price_output:
        clear_output(wait=True)
        selected_token_address = token_address_input.value.strip()
        if not selected_token_address:
            print("❌ Please enter a token address")
            return
            
        print(f"Fetching current price for token: {selected_token_address}...")
        price_data = birdeye.get_token_price(selected_token_address)
        
        if price_data and 'data' in price_data:
            current_price = price_data['data']['value']
            print(f"\n📊 Current Price: ${current_price:.6f}")
            
            # Display additional price info if available
            if 'updateUnixTime' in price_data['data']:
                last_update = datetime.fromtimestamp(price_data['data']['updateUnixTime'])
                print(f"🕐 Last Updated: {last_update.strftime('%Y-%m-%d %H:%M:%S')}")
        else:
            print("❌ Could not fetch price data")

fetch_price_button.on_click(fetch_price)

print("Enter a token address to analyze:")
display(widgets.VBox([token_address_input, fetch_price_button, price_output]))

fetch_price(None)

## Step 4: Get Token Market Data

Now let's get comprehensive token market information. You can enter a different token address if desired:

In [ ]:
# Token address input widget for market data
token_address_input_step4 = widgets.Text(
    value=get_default_token_address(),  # Use selected token or fallback to SOL
    placeholder='Enter token address',
    description='Token Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Create button to fetch market data
fetch_market_button = widgets.Button(
    description='Get Market Data',
    button_style='info',
    tooltip='Click to fetch market data'
)

# Output widget for results
market_output = widgets.Output()

def fetch_market_data(b):
    with market_output:
        clear_output(wait=True)
        selected_address = token_address_input_step4.value.strip()
        if not selected_address:
            print("❌ Please enter a token address")
            return
            
        print(f"Fetching detailed market data for token: {selected_address}...")
        token_market_data = birdeye.get_token_market_data(selected_address)
        
        if token_market_data and 'data' in token_market_data:
            # Display formatted token information
            market_data = token_market_data['data']
            
            print(f"\nToken Market Data:")
            print("=" * 50)
            print(f"Address: {market_data.get('address', 'N/A')}")
            print(f"Price: ${market_data.get('price', 0):.6f}")
            print(f"Market Cap: {format_currency(market_data.get('market_cap', 0))}")
            print(f"FDV: {format_currency(market_data.get('fdv', 0))}")
            print(f"Liquidity: {format_currency(market_data.get('liquidity', 0))}")
            print(f"Total Supply: {market_data.get('total_supply', 0):,.0f}")
            print(f"Circulating Supply: {market_data.get('circulating_supply', 0):,.0f}")
            print("=" * 50)
        else:
            print("❌ Could not fetch token market data")

fetch_market_button.on_click(fetch_market_data)

print("Enter a token address to get market data:")
display(widgets.VBox([token_address_input_step4, fetch_market_button, market_output]))

# Auto-run with selected token address
print(f"\n🚀 Auto-running with selected token: {get_default_token_address()[:8]}...")
fetch_market_data(None)

## Step 5: Create Interactive Price Chart

Let's visualize the token's price history with an interactive chart. Enter a token address to analyze:

In [ ]:
# Token address input widget for price history
# Token address input widget for price history
token_address_input_step5 = widgets.Text(
    value=get_default_token_address(),  # Use selected token or fallback to SOL
    placeholder='Enter token address',
    description='Token Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Create button to fetch price history
fetch_chart_button = widgets.Button(
    description='Get Price Chart',
    button_style='info',
    tooltip='Click to fetch price history chart'
)

# Output widget for results
chart_output = widgets.Output()

def fetch_price_chart(b):
    with chart_output:
        clear_output(wait=True)
        selected_address_step5 = token_address_input_step5.value.strip()
        if not selected_address_step5:
            print("❌ Please enter a token address")
            return
            
        print(f"Fetching price history for token: {selected_address_step5}...")
        price_history = birdeye.get_price_history(selected_address_step5, type_="1H")
        # Create interactive price chart (only show if real data is available)
        price_chart = create_price_chart(price_history, selected_address_step5)
        
        if price_chart:
            print(f"\n📈 Price Chart:")
            price_chart.show()
        else:
            print("\n⚠️ No price chart available - insufficient real data")
            print("This token may be too new or have limited trading history.")

fetch_chart_button.on_click(fetch_price_chart)

print("Enter a token address to get price history:")
display(widgets.VBox([token_address_input_step5, fetch_chart_button, chart_output]))

# Auto-run with selected token address
print(f"\n🚀 Auto-running with selected token: {get_default_token_address()[:8]}...")
fetch_price_chart(None)

## Step 6: Display Recent Transactions

Let's look at the most recent transactions for this token. Enter a token address to analyze:

In [ ]:
# Token address input widget for transactions
token_address_input_step6 = widgets.Text(
    value=get_default_token_address(),  # Use selected token or fallback to SOL
    placeholder='Enter token address',
    description='Token Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

# Create button to fetch transactions
fetch_txs_button = widgets.Button(
    description='Get Transactions',
    button_style='info',
    tooltip='Click to fetch recent transactions'
)

# Output widget for results
txs_output = widgets.Output()

def fetch_transactions(b):
    with txs_output:
        clear_output(wait=True)
        selected_address_step6 = token_address_input_step6.value.strip()
        if not selected_address_step6:
            print("❌ Please enter a token address")
            return
            
        print(f"Fetching recent transactions for token: {selected_address_step6}...")
        recent_txs = birdeye.get_token_transactions(selected_address_step6, limit=20)
        
        if recent_txs and 'data' in recent_txs:
            transactions = recent_txs['data']['items']
            
            if transactions:
                print(f"\nFound {len(transactions)} recent transactions:")
                
                # Create a list to store transaction data for the table
                tx_data = []
                
                for tx in transactions[:10]:  # Show top 10 transactions
                    # Extract transaction data
                    tx_hash = tx.get('tx_hash', 'N/A')
                    block_timestamp = tx.get('block_unix_time', 'N/A')
                    side = tx['side']
                    source = tx.get('source', 'Unknown')
                    
                    # From amount and symbol
                    from_amount = tx['from'].get('ui_amount', 0)
                    from_symbol = tx['from'].get('symbol', 'Unknown')
                    from_str = f"{from_amount:,.4f} {from_symbol}" if from_amount else 'N/A'
                    
                    # To amount and symbol
                    to_amount = tx['to'].get('ui_amount', 0)
                    to_symbol = tx['to'].get('symbol', 'Unknown')
                    to_str = f"{to_amount:,.4f} {to_symbol}" if to_amount else 'N/A'
                    
                    volume_usd = tx.get('volume_usd', 0)
                    volume_str = format_currency(volume_usd) if volume_usd else 'N/A'
                    
                    tx_data.append({
                        'Block Time': datetime.fromtimestamp(block_timestamp),
                        'TX Hash': tx_hash,
                        'From': from_str,
                        'To': to_str,
                        'Side': side,
                        'Volume USD': volume_str,
                        'Source': source,
                    })
                
                # Create and display DataFrame table
                if tx_data:
                    tx_df = pd.DataFrame(tx_data)
                    
                    # Set display options for better table formatting
                    pd.set_option('display.max_columns', None)
                    pd.set_option('display.width', None)
                    pd.set_option('display.max_colwidth', 50)
                    
                    print("\n📊 Recent Transactions Table:")
                    display(tx_df)
                    
                    # Reset display options
                    pd.reset_option('display.max_columns')
                    pd.reset_option('display.width')
                    pd.reset_option('display.max_colwidth')
                else:
                    print("No transaction data to display")
            else:
                print("No recent transactions found")
        else:
            print("❌ Could not fetch transaction data")

fetch_txs_button.on_click(fetch_transactions)

print("Enter a token address to get recent transactions:")
display(widgets.VBox([token_address_input_step6, fetch_txs_button, txs_output]))

# Auto-run with selected token address
print(f"\n🚀 Auto-running with selected token: {get_default_token_address()}")
fetch_transactions(None)

## Step 7: Direct HTTP Requests

Let's see how to make direct HTTP requests without using our wrapper:

In [ ]:
# Example: Direct HTTP request to get token price
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def get_token_price_direct(token_address):
    """Get token price using direct HTTP request"""
    api_key = os.getenv('BDS_STANDARD_API_KEY')
    if not api_key:
        print("❌ API key not found")
        return None
    
    url = "https://public-api.birdeye.so/defi/price"
    headers = {
        'X-API-KEY': api_key,
        'Content-Type': 'application/json'
    }
    params = {'address': token_address}
    
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None

sol_address = "So11111111111111111111111111111111111111112"

print("🔄 Comparing Wrapper vs Direct HTTP Requests:")
print("=" * 60)

# Direct HTTP request
direct_price = get_token_price_direct(sol_address)
if direct_price and 'data' in direct_price:
    print(f"   💰 SOL Price: ${direct_price['data']['value']:.2f}")
    print(f"   ✅ Direct HTTP method successful")

## Step 8: Interactive Token Explorer

Let's create an interactive widget to explore different tokens:

In [ ]:
# Create interactive token explorer
# Output widget for explorer results
explorer_output = widgets.Output()

def explore_token(token_address_input):
    """Explore any token by address"""
    with explorer_output:
        clear_output(wait=True)
        if not token_address_input.strip():
            print("❌ Please enter a token address")
            return
        
        print(f"🔍 Exploring token: {token_address_input}")
        print("-" * 50)
        
        # Get token market data
        market_data = birdeye.get_token_market_data(token_address_input)
        if market_data and 'data' in market_data:
            data = market_data['data']
            print(f"💰 Price: ${data.get('price', 0):.6f}")
            print(f"📊 Market Cap: {format_currency(data.get('market_cap', 0))}")
            print(f"💧 Liquidity: {format_currency(data.get('liquidity', 0))}")
            print(f"🔢 Total Supply: {data.get('total_supply', 0):,.0f}")
        else:
            print("❌ Could not fetch token information")
            return
        
        # Get current price
        time.sleep(1)
        price = birdeye.get_token_price(token_address_input)
        if price and 'data' in price:
            print(f"\n💰 Current Price: ${price['data']['value']:.6f}")
        
        # Create mini price chart
        time.sleep(1)
        print("\n📈 Generating price chart...")
        history = birdeye.get_price_history(token_address_input, type_="1D")
        if history:
            symbol = token_address_input[:8] + '...'  # Use address as symbol fallback
            chart = create_price_chart(history, symbol)
            if chart:
                chart.update_layout(height=300)
                chart.show()
            else:
                print("📊 No price chart available (insufficient data)")
        else:
            print("📊 No price history available")

# Create input widget
token_input = widgets.Text(
    value=get_default_token_address(),  # Use selected token or fallback to SOL
    placeholder='Enter token address (e.g., So11111111111111111111111111111111111111112)',
    description='Token Address:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='600px')
)

explore_button = widgets.Button(
    description='Explore Token',
    button_style='info',
    tooltip='Click to explore the token'
)

def on_explore_click(b):
    explore_token(token_input.value)

explore_button.on_click(on_explore_click)

print("Interactive Token Explorer:")
display(widgets.VBox([token_input, explore_button, explorer_output]))

print("\n💡 Try exploring these popular tokens:")
print("   • SOL: So11111111111111111111111111111111111111112")
print("   • USDC: EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v")
print("   • RAY: 4k3Dyjzvzp8eMZWUXbBCjEvwSkkk59S5iCNLY3QrkX6R")
print("   • JUP: JUPyiwrYJFskUPiHa7hkeR8VUtAeFoSYbKedZNsDvCN")
print("   • USELESS: Dz9mQ9NzkBcCsuGPFJ3r1bS4wgqKMHBPiVuniW8Mbonk")

## 🎉 Congratulations!

You've successfully built a comprehensive token information dashboard! Here's what you've accomplished:

### What You've Learned:
- **Token Discovery**: Fetched a list of tokens using `/defi/tokenlist`
- **Price Data**: Retrieved real-time prices with `/defi/price`
- **Token Analysis**: Got comprehensive market data using `/defi/v3/token/market-data`
- **Visualization**: Created interactive charts with `/defi/history_price`
- **Transaction Tracking**: Displayed recent activity using `/defi/v3/token/txs`

### Next Steps:
Ready for Section 2? Let's build a wallet portfolio manager with real-time WebSocket updates!

**Next**: Open `02_Wallet_Portfolio.ipynb` to continue your journey.

### Challenge Ideas:
- Add more chart types (volume, market cap)
- Implement token comparison features
- Create alerts for price changes
- Build a token watchlist

Great work!